In [8]:
import requests
import pandas as pd
import os
import csv
import time
from typing import Optional

# ESM Atlas API configuration
ESM_API_URL = "https://api.esmatlas.com/foldSequence/v1/pdb/"

def fold_sequence_with_api(sequence: str, max_retries: int = 3, delay: int = 2) -> Optional[str]:
    """
    Fold a protein sequence using ESM Atlas API
    
    Args:
        sequence: Amino acid sequence to fold
        max_retries: Maximum number of retry attempts
        delay: Delay between retries in seconds
    
    Returns:
        PDB content as string, or None if failed
    """
    for attempt in range(max_retries):
        try:
            print(f"  Folding sequence (attempt {attempt + 1}/{max_retries})...")
            
            # Make API request
            response = requests.post(
                ESM_API_URL,
                data=sequence,
                headers={'Content-Type': 'text/plain'},
                timeout=60  # 60 second timeout
            )
            
            if response.status_code == 200:
                print(f"  ✓ Folding successful")
                return response.text
            elif response.status_code == 429:  # Rate limited
                print(f"  Rate limited, waiting {delay * (attempt + 1)} seconds...")
                time.sleep(delay * (attempt + 1))
            else:
                print(f"  API error: {response.status_code} - {response.text}")
                if attempt < max_retries - 1:
                    time.sleep(delay)
                    
        except requests.exceptions.Timeout:
            print(f"  Request timed out (attempt {attempt + 1})")
        except requests.exceptions.RequestException as e:
            print(f"  Request failed: {e}")
            
        if attempt < max_retries - 1:
            time.sleep(delay)
    
    print(f"  ✗ Failed to fold sequence after {max_retries} attempts")
    return None

print("ESM Atlas API setup complete - no local model loading required!")
print("This approach uses the cloud API, avoiding GPU memory issues.")

ESM Atlas API setup complete - no local model loading required!
This approach uses the cloud API, avoiding GPU memory issues.


In [ ]:
# Process sequences using ESM Atlas API instead of local model
csv_file = pd.read_csv("generated_sequences.csv")

# Create headers for the results CSV file if it doesn't exist
if not os.path.exists("invpep-test-final.csv"):
    with open("invpep-test-final.csv", 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['pdb_id', 'sequence', 'pdb_path'])  # Add headers

total_sequences = 0
successful_folds = 0
failed_folds = 0

for _, row in csv_file.iterrows():
    pdb_counter = 0  # Initialize counter starting from 0
    pdb_name = row['pdb_id']
    
    print(f"\n=== Processing peptide: {pdb_name} ===")
    
    # Process all sequence columns (seq_0 through seq_9)
    for col_name in csv_file.columns[1:]:  # Skip only pdb_id column
        if col_name.startswith('seq_'):  # Only process sequence columns
            seq = row[col_name]
            
            # Skip empty sequences
            if pd.isna(seq) or seq == "":
                print(f"Skipping {pdb_name} {col_name}: Empty sequence")
                pdb_counter += 1
                continue
                
            print(f"\nProcessing {pdb_name} with sequence {col_name}: {seq[:50]}... (length: {len(seq)})")
            total_sequences += 1
            
            # Use ESM Atlas API to fold the sequence
            pdb_content = fold_sequence_with_api(seq)
            
            if pdb_content:
                # Save the PDB file with incremented counter
                pdb_path = f"invpep_test_esmfold_final/{pdb_name}/{pdb_name}_{pdb_counter}.pdb"
                os.makedirs(f"invpep_test_esmfold_final/{pdb_name}", exist_ok=True)
                
                with open(pdb_path, 'w') as f:
                    f.write(pdb_content)
                
                print(f"✓ Saved PDB for {pdb_name} {col_name} at {pdb_path}")
                successful_folds += 1
                
                # Add pdb_path to results CSV
                with open("invpep-test-final.csv", 'a', newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([pdb_name, seq, pdb_path])
                
            else:
                print(f"✗ Failed to fold sequence for {pdb_name} {col_name}")
                failed_folds += 1
            
            pdb_counter += 1  # Increment counter for next file
            
            # Add a small delay between API calls to be respectful
            time.sleep(1)

print(f"\n=== PROCESSING COMPLETE ===")
print(f"Total sequences processed: {total_sequences}")
print(f"Successful folds: {successful_folds}")
print(f"Failed folds: {failed_folds}")
print(f"Success rate: {(successful_folds/total_sequences)*100:.1f}%" if total_sequences > 0 else "No sequences processed")


=== Processing peptide: 1a1r_C_peptide ===

Processing 1a1r_C_peptide with sequence seq_0: GSVVIVGRIVLSGKPK... (length: 16)
  Folding sequence (attempt 1/3)...
  ✓ Folding successful
✓ Saved PDB for 1a1r_C_peptide seq_0 at invpep_test_esmfold_final/1a1r_C_peptide/1a1r_C_peptide_0.pdb

Processing 1a1r_C_peptide with sequence seq_1: GSVVIVGRIVLSGKPR... (length: 16)
  Folding sequence (attempt 1/3)...
  ✓ Folding successful
✓ Saved PDB for 1a1r_C_peptide seq_1 at invpep_test_esmfold_final/1a1r_C_peptide/1a1r_C_peptide_1.pdb

Processing 1a1r_C_peptide with sequence seq_2: GSVVIVGRIVLSGKPG... (length: 16)
  Folding sequence (attempt 1/3)...
  ✓ Folding successful
✓ Saved PDB for 1a1r_C_peptide seq_2 at invpep_test_esmfold_final/1a1r_C_peptide/1a1r_C_peptide_2.pdb

Processing 1a1r_C_peptide with sequence seq_3: GSVVIVGRIVLSGKPR... (length: 16)
  Folding sequence (attempt 1/3)...
  ✓ Folding successful
✓ Saved PDB for 1a1r_C_peptide seq_3 at invpep_test_esmfold_final/1a1r_C_peptide/1a1r_C_pe

In [10]:
import subprocess
import os
import pandas as pd
import re
from collections import defaultdict

def run_tm_score_comparison():
    """
    Compare ESMFold generated structures with original structures using TM-score
    and rank sequences based on TM-score values
    """
    
    # Read the generated sequences data
    df = pd.read_csv('generated_sequences.csv')
    
    # Path to original PDB structures
    orig_pdb_path = "original_pdbs"
    
    # Path to ESMFold generated structures (this is where your actual PDBs are)
    esmfold_pdb_path = "invpep_test_esmfold_final"
    
    # Store all results
    all_results = []
    
    # Process each peptide
    for _, row in df.iterrows():
        pdb_name = row['pdb_id']
        print(f"\nProcessing {pdb_name}...")
        
        # Original PDB file path
        orig_pdb = os.path.join(orig_pdb_path, f"{pdb_name}.pdb")
        
        if not os.path.exists(orig_pdb):
            print(f"Warning: Original PDB not found: {orig_pdb}")
            continue
        
        # Results for this peptide
        peptide_results = []
        
        # Check what PDB files actually exist for this peptide
        peptide_folder = os.path.join(esmfold_pdb_path, pdb_name)
        if not os.path.exists(peptide_folder):
            print(f"Warning: ESMFold folder not found: {peptide_folder}")
            continue
        
        # Get all PDB files in the peptide folder
        pdb_files = [f for f in os.listdir(peptide_folder) if f.endswith('.pdb')]
        pdb_files.sort()  # Sort to ensure consistent ordering
        
        print(f"Found {len(pdb_files)} PDB files: {pdb_files}")
        
        # Compare each PDB file with the original
        for pdb_file in pdb_files:
            # Extract sequence index from filename (e.g., peptide_A_0.pdb -> 0)
            try:
                seq_index = int(pdb_file.split('_')[-1].replace('.pdb', ''))
                seq_col = f"seq_{seq_index}"
                
                # Check if this sequence column exists in our CSV
                if seq_col not in df.columns:
                    print(f"  Skipping {pdb_file}: No corresponding sequence column {seq_col}")
                    continue
                
                seq = row[seq_col]
                if pd.isna(seq) or seq == "":
                    print(f"  Skipping {pdb_file}: Empty sequence in {seq_col}")
                    continue
                
            except (ValueError, IndexError):
                print(f"  Skipping {pdb_file}: Cannot parse sequence index")
                continue
            
            # Full path to ESMFold generated PDB
            esmfold_pdb = os.path.join(peptide_folder, pdb_file)
            
            try:

                env = os.environ.copy()
                env["LD_LIBRARY_PATH"] = "/home/ribodiffusion/.conda/envs/ribodiffusionenv/lib:" + env.get("LD_LIBRARY_PATH", "")

                # Run TM-score comparison (Linux command)
                command = subprocess.run(
                    ["./TMscore", orig_pdb, esmfold_pdb], 
                    capture_output=True, 
                    text=True,
                    timeout=60,  # 60 second timeout
                    env=env 
                )
                
                if command.returncode == 0:
                    # Extract TM-score from output
                    tm_score_match = re.search(r"TM-score\s*=\s*([\d.]+)", command.stdout)
                    
                    if tm_score_match:
                        tm_score = float(tm_score_match.group(1))
                        
                        # Store result
                        result = {
                            'pdb_id': pdb_name,
                            'sequence_variant': seq_col,
                            'sequence': seq,
                            'sequence_length': len(seq),
                            'tm_score': tm_score,
                            'orig_pdb_path': orig_pdb,
                            'esmfold_pdb_path': esmfold_pdb,
                            'seq_index': seq_index,
                            'pdb_filename': pdb_file
                        }
                        
                        peptide_results.append(result)
                        all_results.append(result)
                        
                        print(f"  {seq_col} ({pdb_file}): TM-score = {tm_score:.4f}")
                    else:
                        print(f"  {seq_col} ({pdb_file}): Could not extract TM-score")
                else:
                    print(f"  {seq_col} ({pdb_file}): TM-score command failed")
                    
            except subprocess.TimeoutExpired:
                print(f"  {seq_col} ({pdb_file}): TM-score command timed out")
            except Exception as e:
                print(f"  {seq_col} ({pdb_file}): Error running TM-score: {e}")
        
        # Rank sequences for this peptide by TM-score
        if peptide_results:
            peptide_results.sort(key=lambda x: x['tm_score'], reverse=True)
            print(f"\nRanking for {pdb_name}:")
            for i, result in enumerate(peptide_results, 1):
                print(f"  Rank {i}: {result['sequence_variant']} ({result['pdb_filename']}) - TM-score: {result['tm_score']:.4f}")
    
    # Create comprehensive results DataFrame
    if all_results:
        results_df = pd.DataFrame(all_results)
        
        # Sort by pdb_id and then by TM-score (descending)
        results_df = results_df.sort_values(['pdb_id', 'tm_score'], ascending=[True, False])
        
        # Add ranking within each peptide group
        results_df['rank_within_peptide'] = results_df.groupby('pdb_id')['tm_score'].rank(method='dense', ascending=False)
        
        # Save detailed results
        results_df.to_csv('tm_score_detailed_results.csv', index=False)
        print(f"\nDetailed results saved to 'tm_score_detailed_results.csv'")
        
        # Create summary with best sequences
        best_sequences = results_df.groupby('pdb_id').first().reset_index()
        best_sequences.to_csv('best_sequences_by_tm_score.csv', index=False)
        print(f"Best sequences summary saved to 'best_sequences_by_tm_score.csv'")
        
        # Display summary statistics
        print(f"\nSummary Statistics:")
        print(f"Total comparisons: {len(results_df)}")
        print(f"Average TM-score: {results_df['tm_score'].mean():.4f}")
        print(f"Best TM-score: {results_df['tm_score'].max():.4f}")
        print(f"Worst TM-score: {results_df['tm_score'].min():.4f}")
        
        return results_df
    else:
        print("No successful TM-score comparisons completed")
        return None

# Run the comparison
print("Starting TM-score comparison and ranking...")
print("Make sure TMscore executable is in the current directory and has execute permissions")
print("Use: chmod +x TMscore")
print()

results = run_tm_score_comparison()

Starting TM-score comparison and ranking...
Make sure TMscore executable is in the current directory and has execute permissions
Use: chmod +x TMscore


Processing 1a1r_C_peptide...
Found 10 PDB files: ['1a1r_C_peptide_0.pdb', '1a1r_C_peptide_1.pdb', '1a1r_C_peptide_2.pdb', '1a1r_C_peptide_3.pdb', '1a1r_C_peptide_4.pdb', '1a1r_C_peptide_5.pdb', '1a1r_C_peptide_6.pdb', '1a1r_C_peptide_7.pdb', '1a1r_C_peptide_8.pdb', '1a1r_C_peptide_9.pdb']
  seq_0 (1a1r_C_peptide_0.pdb): Could not extract TM-score
  seq_1 (1a1r_C_peptide_1.pdb): Could not extract TM-score
  seq_2 (1a1r_C_peptide_2.pdb): Could not extract TM-score
  seq_3 (1a1r_C_peptide_3.pdb): Could not extract TM-score
  seq_4 (1a1r_C_peptide_4.pdb): Could not extract TM-score
  seq_5 (1a1r_C_peptide_5.pdb): Could not extract TM-score
  seq_6 (1a1r_C_peptide_6.pdb): Could not extract TM-score
  seq_7 (1a1r_C_peptide_7.pdb): Could not extract TM-score
  seq_8 (1a1r_C_peptide_8.pdb): Could not extract TM-score
  seq_9 (1a1r_C_peptide_9.